In [1]:
import json
import requests
from bs4 import BeautifulSoup

url = "https://170-leaderboard.vercel.app/team/discordggTYsy64VcWT"
page = requests.get(url)
data = BeautifulSoup(page.content, 'html.parser')

def get_team_json_table():
    json_table = data.find_all('script')[9].string
    json_table = json.loads(json_table)
    json_table = json_table["props"]["pageProps"]["data"]
    return json_table

def get_result_json_table():
    json_table = data.find_all('script')[9].string
    json_table = json.loads(json_table)
    json_table = json_table["props"]["pageProps"]["sortedData"]
    return json_table

def expand_input(name):
    if name.startswith('large'):
        return ('large', int(name[5:-3]))
    elif name.startswith('medium'):
        return ('medium', int(name[6:-3]))
    elif name.startswith('small'):
        return ('small', int(name[5:-3]))

class Result:
    size = None
    test = None
    rank = None
    best_score = None
    url = None

    def __init__(self, name, rank, score):
        self.size, self.test = expand_input(name)
        self.rank = rank
        self.best_score = score
        self.url = f"https://170-leaderboard.vercel.app/input/{self.size}/{self.test}"

    def __str__(self):
        return f"{self.size} {self.test} {self.rank} {self.best_score} {self.url}"
    
    def __repr__(self):
        return f"{self.size} {self.test} {self.rank} {self.best_score} {self.url}"

team_json_table = get_team_json_table()
table = dict()
for key in team_json_table:
    table[key] = (team_json_table[key]["score"], team_json_table[key]["rank"])

sorted_table = []
for key in table:
    sorted_table.append(Result(key, table[key][1], table[key][0]))
sorted_table.sort(key=lambda x: x.rank, reverse=True)

with open("../queue.txt", "w") as f:
    f.write("")

for result in sorted_table:
    if result.rank != 1:
        page = requests.get(result.url)
        data = BeautifulSoup(page.content, 'html.parser')
        result_json_table = get_result_json_table()
        result.best_score = result_json_table[0]["score"]
    print(result, file=open("../queue.txt", "a"))